In [32]:
import cv2 as cv
import sys
sys.path.append("..")
import slam_utils as su
import numpy as np
import time

In [33]:
# load images
img_1 = cv.imread("./1.png")
img_2 = cv.imread("./2.png")
keypoints_1, keypoints_2, matches = su.find_feature_matches(img_1, img_2)
print(f"Number of matches: {len(matches)}")

d1 = cv.imread("./1_depth.png", cv.IMREAD_UNCHANGED)
K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])


Max dist: 94.0
Min dist: 4.0
Number of matches: 79


In [34]:
# these have to be Nx3 and Nx2 numpy arrays
pts_3d = np.empty((0, 3))
pts_2d = np.empty((0, 2))
for m in matches:
    d = d1[int(keypoints_1[m.queryIdx].pt[1]),
           int(keypoints_1[m.queryIdx].pt[0])]
    if d == 0:
        continue
    dd = d/5000.0
    p1 = su.pixel2cam(keypoints_1[m.queryIdx].pt, K)
    pts_3d = np.vstack([pts_3d, np.array([p1[0]*dd, p1[1]*dd, dd])])
    pts_2d = np.vstack([pts_2d, keypoints_2[m.trainIdx].pt])
print(f"3d-2d pairs: {len(pts_3d)}")

3d-2d pairs: 75


In [35]:
t1 = time.time()
temp, r, t = cv.solvePnP(pts_3d, pts_2d, K, np.array([]))
R = cv.Rodrigues(r)[0]
t2 = time.time()
print(f"Solved pnp in opencv time: {t2-t1}")
print(f"R = {R}")
print(f"t = {t}")

Solved pnp in opencv time: 0.0007300376892089844
R = [[ 0.99790591 -0.0509194   0.03988747]
 [ 0.04981866  0.99836232  0.02812094]
 [-0.04125405 -0.02607491  0.99880839]]
t = [[-0.12678213]
 [-0.00843949]
 [ 0.06034935]]
